In [2]:
from astropy.time import Time
import datetime as dt
import astropy.units as u
import warnings
from astropy.utils.exceptions import AstropyUserWarning, AstropyWarning
from astropy.table import vstack
import numpy as np

# Import astroplan tools and constraints
# Other constraints are available
from astroplan import (FixedTarget, Observer, EclipsingSystem,
                       PrimaryEclipseConstraint, is_event_observable,
                       AtNightConstraint, AltitudeConstraint,
                       LocalTimeConstraint, AirmassConstraint)

from masterfile import MasterFile

# Get planets infos

In [3]:
target_list = ["GJ 436 b",
               "GJ 1214 b",
               "GJ 3470 b",
               "K2-25 b",
               "HD 189733 b",
               "WASP-69 b",
               "HAT-P-1 b",
               "WASP-80 b",
               "WASP-107 b"]

In [5]:
data = MasterFile.read().by_plName(*target_list)

# Inputs

In [7]:
t1, t2 = Time(['2019-02-01 00:00', '2019-07-31 00:00'])
site = 'cfht'
baseline = 2
# See astroplan available constraints
constraints = [AtNightConstraint.twilight_nautical(),
               AirmassConstraint(max=2)
              ]
supp_cols = ['st_j']
n_eclipses = 5000

## Test if all required infos are there

In [33]:
required_info = data['pl_name','pl_tranmid','pl_orbper','pl_trandur']

In [43]:
print('Add missing infos with MasterFile.insert_value()!!')
required_info.show_in_notebook()

Add missing infos with MasterFile.insert_value()!!


idx,pl_name,pl_tranmid,pl_orbper,pl_trandur
,,d,d,d
0,GJ 436 b,2454510.80162,2.64388312,--
1,GJ 1214 b,2455320.535733,1.58040456,--
2,GJ 3470 b,2455983.70421,3.3366496,--
3,K2-25 b,2457062.57935,3.484552,0.0329167
4,HD 189733 b,2453955.5255511,2.21857567,0.07515
5,WASP-69 b,--,3.86814,--
6,HAT-P-1 b,2453979.93202,4.46529976,0.11875
7,WASP-80 b,2456487.425006,3.06785234,0.0887917
8,WASP-107 b,2456514.4106,5.72149,0.1147


## Define some quantities

In [9]:
# Define needed quantities based on planets infos
# Must be quatities arrays (astropy)
# Here we use a given astropy Table (data) to get the infos

epoch, period, transit_duration = data.cols_2_qarr('pl_tranmid','pl_orbper','pl_trandur')
epoch = Time(epoch, format='jd')
pl_name = data['pl_name']


observing_time = t1
obs = Observer.at_site(site)
epoch = Time(epoch, format='jd')

In [10]:
transit_duration

<Quantity [0.       , 0.       , 0.       , 0.0329167, 0.07515  ,
           0.       , 0.11875  , 0.0887917, 0.1147   ] d>

# Compute events

In [ ]:
full_table = Table()
for itar in range(len(pl_name)):

    # -------------------------
    # Steps to predict transits
    # -------------------------

    target = FixedTarget.from_name(pl_name[itar])

    sys = EclipsingSystem(primary_eclipse_time=epoch[itar],
                          orbital_period=period[itar],
                          duration=transit_duration[itar],
                          name=target.name
                         )
    sys_baseline = EclipsingSystem(primary_eclipse_time=epoch[itar],
                                   orbital_period=period[itar],
                                   duration=baseline*transit_duration[itar],
                                   name=target.name
                                  )

    t_mid = sys.next_primary_eclipse_time(observing_time, n_eclipses=n_eclipses)

    if t_mid[-1] < t2:
        warnings.warn('end time ('+t2.value +
                      ') is passed the last computed event time (' +
                      t_mid[-1].value+')\n' +
                     '\t You can change the n_eclipse kwarg ' +
                     'value or choose a different window (start or end time)',
                     AstropyUserWarning
                    )

    # You can skip this step by addind a time constraint in the list of constraints
    t_mid = t_mid[t_mid < t2]

    n_event, = t_mid.shape

    t_baseline = sys_baseline.next_primary_ingress_egress_time(observing_time, n_eclipses=n_event)

    t1_t4 = sys.next_primary_ingress_egress_time(observing_time, n_eclipses=n_event)

    index = is_event_observable(constraints_list, obs, target,
                                times_ingress_egress=t_baseline).squeeze()

    # -------------------
    # End of steps to predict transits
    # -------------------

    # Put the infos in a table and stack it to the full table
    if index.any():
        name = np.repeat(sys.name,index.sum()).astype(str)
        moon = obs.moon_illumination(t_mid[index])
        AM_mid = obs.altaz(t_mid[index], target).secz
        AM_t1_t4 = obs.altaz(t1_t4[index], target).secz
        AM_base = obs.altaz(t_baseline[index], target).secz
        supp = [np.repeat(data[key][itar],index.sum())
                for key in supp_cols]
        cols = [name,
                t_mid[index].iso,
                AM_mid,
                *t1_t4[index].T.iso,
                *AM_t1_t4.T,
                *t_baseline[index].T.iso,
                *AM_base.T,
                moon,
                *supp
               ]
        table_sys = Table(cols, names=names, masked=True)
        full_table = vstack([table_sys, full_table])
    else:
        warnings.warn('No event found for '+sys.name, AstropyUserWarning)

full_table.sort('mid_tr')
full_table.meta = meta

## Save file

In [ ]:
full_table.write('list_of_transits.ecsv', delimiter=',')